In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')
import seaborn as sns
sns.set_context("paper")
sns.set_style("ticks");

# Discovering and forecasting extreme events via active learning in neural operators

Citation - Pickering, E., Guth, S., Karniadakis, G. E. & Sapsis, T. P. Discovering and forecasting extreme events via active learning in neural operators. Nat Comput Sci 2, 823–833 (2022).

## 2) MMT Nonlinear Wave

* We seek to map initially observed waves, $u(x, t = 0)$, where $x$ is the spatial variable and $t$ is time, to the QoI: the future spatial maximum $G(x) = ||Re(u(x, t = T; x))||_{\infty}$, where $T$ is a prescribed prediction horizon.
* MMT has complex solutions and therefore the ICs are also complex-valued.
* Dispersive nonlinear wave equation used for studying 1D wave turbulence.
$$
iu_{t} = |\partial_{x}|^{\alpha} u + \lambda |\partial_{x}|^{-\beta/4} \Big(||\partial_{x}|^{- \beta /4} u \Big|^{2} |\partial_{x}|^{- \beta /4} u \Big) + iDu,
$$
* where $u$ is a complex scalar, 
* exponents $\alpha$ and $\beta$ are chosen model parameters, 
* and $D$ is a selective Laplacian
* This model gives rise to four-wave resonant interactions that, especially when coupled with large scale forcing and small scale damping, produces a family of spectra revealing both direct and inverse cascades.
* Transform the equation into wavenumber space.
* The pseudodifferential operator $|\partial_{x}|^{\alpha}$, via the Fourier transform in space becomes: $\widehat{|\partial_{x}|^{\alpha} u(k)} = |k|^{\alpha} \widehat{u} (k)$ where $k$ is the wavenumber in $x$. 
* This formulation may be similarly defined on a periodic domain. 
* We choose $\alpha = 1/2$ and $\beta = 0$. Above equation reduces to
$$
\hat{u}(k)_{t} = -i |k|^{1/2} \hat{u} (k) - i \lambda |\hat{u} (k)|^{2} \hat{u} (k) + \widehat{Du}(k) + f(k)
$$
* where $f(k)$ is a forcing and $\widehat{Du}(k)$ is a selective Laplacian of the form:
$$
\widehat{Du}(k) = 
\begin{cases}
  -(|k| - k^{*})^2 \hat{u} k, & |k|>k^{*} \\
  0, & |k| \leq k^{*} \\
\end{cases}
$$
* where $k^{*}$ presents the lower bound of wavenumbers subject to dissipation.
* For the model considered in this study we choose $\lambda = −0.5, k^{*} = 20, f (k) = 0, dt = 0.001$, and a grid that is periodic between $0-1$ with $N_x = 512$ grid points.
* To propose a stochastic and complex initial condition, $u(x, t = 0)$, we take the complex-valued kernel
$$
k(x,x') = \sigma^{2}_{u} e^{i(x-x')} e^{-\frac{(x-x')^{2}}{\ell_{u}}}
$$
* with $\sigma^{2}_{u} = 1$ and $\ell_{u} = 0.35$.
* We then parametrize the stochastic initial conditions by a finite number of random variables, $x$, using the Karhunen-Loeve expansion of the kernel’s correlation matrix,
$$
u(x,t = 0) \approx \textbf{x} \Phi(x), \space \forall \space x \in [0,1)
$$
* where $x \in \mathbb{C}^m$ is a vector of complex coefficients and both the real and imaginary components of each coefficient are normally distributed with zero mean and diagonal covariance matrix $\Lambda$.
* and $\{\Lambda, \Phi (x)\}$ contains the first $m$ eigenpairs of the correlation matrix.
* This gives the dimension of the parameter space as $2m$ to account for the real and imaginary components of each coefficient. 
* The presented $2D, 4D, 6D, 20D$ results correspond to $m = 1, 2, 3, 10$. 
* For all cases, the random variable $x_i$ is restricted to a domain ranging from -6 to 6, in that 6 standard deviations in each direction from the mean.

In [2]:
from utils.blackbox import BlackBox
from utils.complex_noise import Noise_MMT
from utils.gaussian_input import GaussianInputs
import scipy.io as sio

seed = 1234
iter_num = 2
n_init = 7
init_method = "lhs"
acq = "lhs"
lam = 5.0
N = 15
batch_size = 6
model = "GP"


In [3]:
tf = 1
rank = 3
noise = Noise_MMT([0, tf], rank)

In [4]:
# Create the Theta_u to U map for saving localling to be called by Matlab MMT files
def Save_U(Theta,nsteps,rank):
    y0 = np.zeros((1,nsteps),dtype=np.complex_)
    xr = Theta[0:rank]
    xi = Theta[rank:(2*rank)]
    x = xr + 1j*xi
    y0 = noise.get_sample(x)
    return y0

# Mapping Theta to U with only one input
def map_def(theta_rank):
    rank = int(theta_rank[1])
    theta = theta_rank[0].reshape(rank*2,)
    nsteps = 512
    y0 = Save_U(theta,nsteps,rank)
    return y0

In [ ]:
if iter_num == 0:
        ndim = rank*2
        np.random.seed(seed)
        noise_var = 0
        nsteps = 512
        my_map = BlackBox(map_def, noise_var=noise_var)

        mean, cov = np.zeros(ndim), np.ones(ndim)
        domain = [ [-6, 6] ] * ndim
        inputs = GaussianInputs(domain, mean, cov)
        Theta = inputs.draw_samples(n_init, init_method)
        
        save_y0_file = './data/'+'_Seed'+str(seed)+'_ndim'+str(ndim)+'_rank'+str(rank)+'_n_init'+str(n_init)+'_init_method_'+str(init_method)+'_savedata_y0.mat'

        y0 = np.zeros((nsteps,n_init),dtype=np.complex_)
        for i in range(0,n_init):
            y0[:,i] = Save_U(Theta[i,:],nsteps,rank).reshape(nsteps,)
            # Save the y0 file to run with matlab
        sio.savemat(save_y0_file, {'y0':y0, 'Theta':Theta})        
else:
    ndim = rank*2
    np.random.seed(seed)
    noise_var = 0
    my_map = BlackBox(map_def, noise_var=noise_var)

    mean, cov = np.zeros(ndim), np.ones(ndim)
    domain = [ [-6, 6] ] * ndim
    inputs = GaussianInputs(domain, mean, cov)
    
    # Need to determine U
    nsteps = 512 # Reset the number of steps for deeponet
    x_vals = np.linspace(0, 1, nsteps+1)
    x_vals = x_vals[0:-1]
        
    #Y = my_map.evaluate(Theta,1)    
    coarse = 4
    # Create the X to U map, which is actually theta to U
    def Theta_to_U(Theta,nsteps,coarse,rank):
        # We can also coarsen the steps 512 is likely extra fine for Deeponet
        Theta = np.atleast_2d(Theta)
        U = np.zeros((np.shape(Theta)[0],2*int(nsteps/coarse)),dtype=np.complex_)

        # Determine real and imaginary inds
        dim = int(np.shape(Theta)[1]/2)
        xr = Theta[:,0:(dim)]
        xi = Theta[:,dim:dim*2]
        x = xr + 1j*xi
        Us = np.transpose(noise.get_sample(x))
        coarser_inds = np.linspace(0,nsteps-1,int(nsteps/coarse)).astype(int)

        real_inds = np.linspace(0,nsteps/coarse*2-2,int(nsteps/coarse)).astype(int)
        imag_inds = np.linspace(1,nsteps/coarse*2-1,int(nsteps/coarse)).astype(int)
        
        U[:,real_inds] = np.real(Us[:,coarser_inds])
        U[:,imag_inds] = np.imag(Us[:,coarser_inds])
        return U
    
    def Theta_to_Z(Theta,rank):
        Z = np.ones((Theta.shape[0], 1))
        return Z

    save_path_data_prev = './results/Rank'+str(rank)+'_'+model+'_'+acq+'_Seed'+str(seed)+'_N'+str(N)+'_Batch_'+str(batch_size)+'_Init_'+init_method+'_Iteration'+str(iter_num-1)+'.mat'
    load_Y_file = './IC/Rank'+str(rank)+'_'+model+'_Seed'+str(seed)+'_Acq'+acq+'_Iter'+str(iter_num-1)+'_Lam'+str(lam)+'_BatchSize'+str(batch_size)+'_N'+str(N)+'_savedata_Y.mat' 
    save_y0_file_prev = './IC/Rank'+str(rank)+'_'+model+'_Seed'+str(seed)+'_Acq'+acq+'_Iter'+str(iter_num-1)+'_Lam'+str(lam)+'_BatchSize'+str(batch_size)+'_N'+str(N)+'_savedata_y0.mat'       

    if iter_num == 1:
            d = sio.loadmat(save_y0_file_prev)
            Theta = d['Theta']
            d = sio.loadmat(load_Y_file)
            Y = d['Y']
            print(np.shape(Y))
            
    else: 
        d = sio.loadmat(save_path_data_prev)
        Theta = d['Theta']
        Y = d['Y']
    
        # Load in the files
        d = sio.loadmat(save_y0_file_prev)
        theta_opt = d['theta_opt']
        d = sio.loadmat(load_Y_file)
        Y_opt = d['Y']
    
        Theta = np.vstack((Theta, theta_opt))
        Y = np.vstack((Y, Y_opt))
        print(np.shape(Theta))
        print(np.shape(Y))
    
    np.random.seed(np.size(Y))

    # These functions are defined for normalizing, standardizing, or flatenining interal to DeepONet
    def DNO_Y_transform(x):
        x_transform = x
        return x_transform

    def DNO_Y_itransform(x_transform):
        x = x_transform
        return x
    
    # Train the model
    np.random.seed(np.size(Y)) # Randomize the seed based on Y size for consistency

In [ ]:
# Transform to U and G values       
u_train = Theta_to_U(Theta,nsteps,coarse,rank)
print(f"Shape of u_train:{u_train.shape}")
y_train = Theta_to_Z(Theta,rank)
print(f"Shape of y_train:{y_train.shape}")
G_train = DNO_Y_transform(Y)
print(f"Shape of G_train:{G_train.shape}")

# Validation File

In [ ]:
val_file = sio.loadmat("./data/Rank1_Xs1.mat")
val_file.keys()